In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline

In [ ]:
data  = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/00396/Sales_Transactions_Dataset_Weekly.csv")
data_input = data.filter(regex=r'Product|W')

In [ ]:
data_input.head()

,Product_Code,W0,W1,W2,W3,W4,W5,W6,W7,W8,...,W42,W43,W44,W45,W46,W47,W48,W49,W50,W51
0,P1,11,12,10,8,13,12,14,21,6,...,4,7,8,10,12,3,7,6,5,10
1,P2,7,6,3,2,7,1,6,3,3,...,2,4,5,1,1,4,5,1,6,0
2,P3,7,11,8,9,10,8,7,13,12,...,6,14,5,5,7,8,14,8,8,7
3,P4,12,8,13,5,9,6,9,13,13,...,9,10,3,4,6,8,14,8,7,8
4,P5,8,5,13,11,6,7,9,14,9,...,7,11,7,12,6,6,5,11,8,9


In [ ]:
!pip install --upgrade mxnet==1.6.0
!pip install gluonts

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.7/68.7 MB 10.2 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.3 MB/s eta 0:00:00


In [ ]:
from gluonts.mx import DeepAREstimator, Trainer

/usr/local/lib/python3.9/dist-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(


In [ ]:
ts_code = data_input['Product_Code'].astype('category').cat.codes.values

In [ ]:
df_train = data_input.iloc[:,1:42].values
df_test = data_input.iloc[:,42:].values

In [ ]:
print(df_train.shape, df_test.shape)

(811, 41) (811, 11)


In [ ]:
freq= '1D'
prediction_length = 6

In [ ]:
estimator = DeepAREstimator( freq = freq,
                            context_length = 6,
                            prediction_length = prediction_length,
                            use_feat_static_cat = True,
                            cardinality = [1],
                            num_layers= 2,
                            num_cells= 32,
                            cell_type='lstm',
                            trainer = Trainer(epochs=5)
                            )

In [ ]:
start_train = pd.Timestamp("2011-01-01 ", freq = freq) 
start_test = pd.Timestamp("2014-11-10 ", freq = freq) 

<ipython-input-11-c429057742e5>:1: FutureWarning: The 'freq' argument in Timestamp is deprecated and will be removed in a future version.
  start_train = pd.Timestamp("2011-01-01 ", freq = freq)
<ipython-input-11-c429057742e5>:2: FutureWarning: The 'freq' argument in Timestamp is deprecated and will be removed in a future version.
  start_test = pd.Timestamp("2014-11-10 ", freq = freq)


In [ ]:
from gluonts.dataset.common import ListDataset
from gluonts.dataset.field_names import FieldName

train_ds = ListDataset(
    [
        {
            FieldName.TARGET: target,
            FieldName.START: start_train,
            FieldName.FEAT_STATIC_CAT: fsc
        }for(target,fsc) in zip(df_train[0:7],ts_code[0:7].reshape(-1,1))
    ],
    freq = freq
)

test_ds = ListDataset(
    [
        {
            FieldName.TARGET: target,
            FieldName.START: start_test,
            FieldName.FEAT_STATIC_CAT: fsc
        }for(target,fsc) in zip(df_test,ts_code.reshape(-1,1))
    ],
    freq = freq
)

In [ ]:
next(iter(df_train))

array([ 0, 11])

In [ ]:
predictor = estimator.train(training_data = train_ds)

100%|██████████| 50/50 [00:01<00:00, 28.76it/s, epoch=5/5, avg_epoch_loss=2.4]


In [ ]:
from gluonts.evaluation.backtest import make_evaluation_predictions
forecast_it, ts_it = make_evaluation_predictions(
    dataset=test_ds,
    predictor = predictor,
    num_samples=100
)

In [ ]:
from tqdm.autonotebook import tqdm
print("Obtaining time Series conditioning values...") 
#forecasts = list(forecast_it)
tss = list(tqdm(ts_it,total=len(df_test)))
print("Obtaining time Series predictions...") 
forecasts = list(tqdm(forecast_it,total=len(df_test)))
#forecast_entry=  forecasts[0]
#ts_entry = tss[0]
#forecast_entry

Obtaining time Series conditioning values...


<ipython-input-16-c53a4e58b97b>:1: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


  0%|          | 0/811 [00:00<?, ?it/s]

Obtaining time Series predictions...


  0%|          | 0/811 [00:00<?, ?it/s]

In [ ]:
def plot_prob_forecasts(ts_entry, forecast_entry):
    plot_length = prediction_length
    prediction_intervals = (80.0, 95.0)
    legend = ["observations", "median prediction"] + [
        f"{k}% prediction interval" for k in prediction_intervals
    ][::-1]

    fig, ax = plt.subplots(1, 1, figsize=(10, 7))
    ts_entry[-plot_length:].plot(ax=ax)  # plot the time series
    forecast_entry.plot(prediction_intervals=prediction_intervals, color="g")
    plt.grid(which="both")
    plt.legend(legend, loc="upper left")
    plt.show()


In [ ]:
from gluonts.evaluation import Evaluator
evaluator = Evaluator(quantiles=[0.1,0.5, 0.9])
agg_metrics, item_metrics = evaluator(tss, forecasts)

Running evaluation: 811it [00:00, 17867.51it/s]
/usr/local/lib/python3.9/dist-packages/pandas/core/dtypes/cast.py:1181: UserWarning: Warning: converting a masked element to nan.
  return arr.astype(dtype, copy=True)


In [ ]:
item_metrics

,item_id,forecast_start,MSE,abs_error,abs_target_sum,abs_target_mean,seasonal_error,MASE,MAPE,sMAPE,ND,MSIS,QuantileLoss[0.1],Coverage[0.1],QuantileLoss[0.5],Coverage[0.5],QuantileLoss[0.9],Coverage[0.9]
0,None,2014-11-15,11.927718,17.691442,43.0,7.166667,3.25,0.907253,0.541875,0.411246,0.411429,4.503806,7.772953,0.166667,17.691443,0.666667,5.565960,0.833333
1,None,2014-11-15,4.704934,12.439376,17.0,2.833333,2.50,0.829292,1.064365,0.894320,0.731728,3.403110,3.126133,0.333333,12.439377,0.500000,3.928975,0.833333
2,None,2014-11-15,12.225704,12.295147,52.0,8.666667,4.50,0.455376,0.182487,0.233520,0.236445,8.426435,6.930438,0.000000,12.295147,0.166667,12.133531,0.833333
3,None,2014-11-15,13.992845,15.596150,51.0,8.500000,3.00,0.866453,0.253390,0.317264,0.305807,12.237361,6.641095,0.000000,15.596150,0.166667,11.876127,0.833333
4,None,2014-11-15,6.281822,13.096783,45.0,7.500000,3.50,0.623656,0.345628,0.284980,0.291040,4.434824,3.869824,0.166667,13.096783,0.500000,6.219181,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
806,None,2014-11-15,0.654616,3.699890,3.0,0.500000,0.50,1.233297,0.756841,1.743380,1.233297,11.777089,0.889644,0.000000,3.699890,0.666667,2.864421,0.666667
807,None,2014-11-15,2.233396,8.520752,30.0,5.000000,1.75,0.811500,0.276518,0.328921,0.284025,5.281535,4.726792,0.000000,8.520752,0.000000,1.945379,0.833333
808,None,2014-11-15,3.311262,9.561047,9.0,1.500000,1.25,1.274806,0.710468,1.556986,1.062338,9.112210,2.649013,0.166667,9.561047,0.500000,7.016025,0.666667
809,None,2014-11-15,0.776867,3.380044,3.0,0.500000,0.25,2.253363,0.907479,1.891337,1.126681,56.448878,0.837275,0.000000,3.380045,0.666667,4.541428,0.666667


In [ ]:
item_metrics.MAPE.mean()

0.5292079500899645